REGLAS DE INFERENCIA


In [14]:
def importacion[P, Q](q: Q): P => Q =
  (_: P) => q

def exportacion[P, Q](sipeq: P => Q): P => Q =
  (p: P) => sipeq(p)

def conmutatividad_conjuncion [P, Q](p: P, q: Q): (Q, P) =
  (q, p)

def conmutatividad_disyuncion[P, Q](e: Either[P, Q]): Either[Q, P] =
  e match {
    case Left(p) => Right(p)
    case Right(q) => Left(q)
  }

def distribucion[P, Q, R](e: Either[P, (Q, R)]): (Either[P, Q], Either[P, R]) =
  e match {
    case Left(p) => (Left(p), Left(p))
    case Right((q, r)) => (Right(q), Right(r))
  }


def alternativa[P, Q](p1: Either[P, Q], p2: Not[P]): Q = {
  p1 match {
    case Left(p) => p2(p)
    case Right(q) => q 
  }
}

def rtr[P,Q,R] (p1: P=>Q, p2: Q=>R): P=>R =
    p=> p2(p1(p))

def modusTollens[P, Q](pToQ: P => Q)(notQ: Not[Q]): Not[P] =
  (p: P) => notQ(pToQ(p))

def modusTollensNOT[P, Q](EM: ExcludedMiddle)(p1: Not[P] => Not[Q])(p2:Q):P = {
    EM[P] match {
        case Left(p) => p
        case Right(np) => 
            val nq = p1(np)
            nq(p2)
    }
}

def rImp[P,Q,R](p1: P=>(Q=>R)):(P,Q)=>R=
    (p,q)=>p1(p)(q)

def rExp[P,Q,R](p1: ((P,Q))=>R): P=>(Q=>R)=
    p=>(q=> p1((p,q)))

def rcontr1[P,Q](p1:P=>Q): Not[Q] => Not[P]=
    nq=> modusTollens(p1)(nq)

def rcontr1_1[P,Q](DN: DoubleNegation)(p1:Not[P]=>Q): Not[Q] => P=
    nq=> DN(modusTollens(p1)(nq))

def rcontr1_2[P,Q](p1:P=>Not[Q]): Q => Not[P]=
    q=> { val nnq= DN2(q)
    modusTollens(p1)(nnq)}

def rcontr2[P, Q](DN: DoubleNegation)(p1: Not[Q] => Not[P]): P => Q = 
    p => DN.apply[Q](nq => p1(nq)(p)) 

def ria1[P,Q](p1: Either[P,Q], p2:Not[P]):Q=
    p1 match{
        case Left(p) => p2(p)
        case Right(q)=> q

    }
def ria1_1[P,Q](p1: Either[Not[P],Q], p2:P):Q=
    p1 match{
        case Left(np) => np(p2)
        case Right(q)=> q

    }

def ria2[P,Q](p1: Either[P,Q], p2:Not[Q]):P=
    p1 match{
        case Left(p)=>p
        case Right(q) => p2(q)
    }
def ria2_2[P,Q](p1: Either[P,Not[Q]], p2:Q):P=
    p1 match{
        case Left(p)=>p
        case Right(nq) => nq(p2)
    }


def rDf1[P,Q](p1: Not[Either[P,Q]]):(Not[P], Not[Q])={
    val np: Not[P] = p=> p1(Left(p))
    val nq: Not[Q] = q=> p1(Right(q))
    (np,nq)
}

def rDf1_1[P,Q](EM: ExcludedMiddle)(p1: Not[Either[Not[P],Not[Q]]]):(P, Q)={
    val p = EM[P] match {
                case Left(p) => p
                case Right(np) => 
                val enp: Either[Not[P], Not[Q]] = Left(np)
                p1(enp)
              }
    val q = EM[Q] match {
                case Left(q) => q
                case Right(nq) => 
                val enq: Either[Not[P], Not[Q]] = Right(nq)
                p1(enq)
              }
  (p, q)
}

def rDf1_2[P,Q](EM: ExcludedMiddle)(p1: Not[Either[Not[P],Q]]):(P, Not[Q])={
    val p: P = EM[P] match {
    case Left(p) => p
    case Right(np) => 
      val either = Left(np)
      p1(either)
    }

    val nq: Not[Q] = q => {
    val either = Right(q)
        p1(either)
    }
  (p, nq)
}

def rDf1_And[P, Q](EM: ExcludedMiddle)(p1:Not[(P, Q)]) : Either[Not[P], Not[Q]] = { 
    EM[P] match {
        case Left(p) => Right((q) => p1((p, q)))
        case Right(np) => Left((p => np(p)))
    }
}

def rDf2[P,Q](p1:(Not[P], Not[Q])): Not[Either[P,Q]]=
 e=> e match{
     case Left(p)=> p1._1(p)
     case Right(q)=> p1._2(q)
 }

def rDf3_v1[P,Q](EM: ExcludedMiddle)(p1:Not[P]=>Q ): Either[P,Q]={
    EM[P] match {
    case Left(p) => Left(p)
    case Right(np) => Right(p1(np))
  }   
}

def rDf3_v2[P,Q] (p1:Either[P,Q]): Not[P]=>Q=
    np=> p1 match{
        case Left(p)=>np(p)
        case Right(q)=>q
    }

def rDfAnd[P,Q] (p1:(P,Q)): Not[P=>Not[Q]]=
    aux=> aux(p1._1)(p1._2)

def rDfAnd2[P,Q] (p1:(P,Q)): Not[Either[Not[P],Not[Q]]]=
    e => e match{
        case Left(np) => np(p1._1)
        case Right(nq) => nq(p1._2)
    }

def rDfEither[P,Q](p1: Either[P,Q]): Not[(Not[P],Not[Q])]=
    t => p1 match{
        case Left(p) => t._1(p)
        case Right(q) => t._2(q)
    }

def rdfNotAnd [P,Q](EM: ExcludedMiddle)(p1:Not[(P,Q)]): Either[Not[P],Not[Q]]={
   EM[P] match {
    case Left(p) =>
      Right((q: Q) => p1((p, q)))
    case Right(np) =>
      Left(np)
  }
}

def rdfNotAndImp[P,Q] (EM: ExcludedMiddle)(p1:Not[(P,Not[Q])]): P=>Q = {
  (p: P) => EM[Q] match {
    case Left(q) => q
    case Right(nq) =>
      val np: Not[P] = (p: P) => p1((p, nq))
      np(p)
  }
}

def rdfNotAndImp2 [P,Q](EM: ExcludedMiddle)(p1:Not[Not[P]=>Q]):(Not[P],Not[Q])={
  EM[P] match {
    case Left(p) =>
      val nq: Not[Q] = (q:Q) => p1((np: Not[P]) => np(p))
      val np: Not[P] = (p:P) => p1((np: Not[P]) => np(p))
      (np, nq)
    case Right(np) =>
      val nq: Not[Q] = (q: Q) => p1((np: Not[P]) => q)
      (np, nq)
  }
}

def rDfImp[P,Q](p1: P=>Q): Not[(P,Not[Q])]=
    t=> {val q = p1(t._1)
        t._2(q)}

def reCoimp1[P,Q](p1:P<=>Q, p2:P):Q=
    p1._1(p2)

def reCoimp2[P,Q](p1:P<=>Q, p2:Not[Q]):Not[P]=
    modusTollens(p1._1)(p2)

def reCoimp3[P,Q](p1:P<=>Q, p2:Q):P=
    p1._2(p2)

def reCoimp4[P,Q](p1:P<=>Q, p2:Not[P]):Not[Q]=
    modusTollens(p1._2)(p2)

trait DoubleNegation {
  def apply[P](nnp: Not[Not[P]]): P
}

trait ExcludedMiddle { 
    def apply[P]: Either[P, Not[P]]
}

def DN2[P](p: P): Not[Not[P]] = //P=>Nothing=>Nothing
     np=> np(p)

type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)

defined function importacion
defined function exportacion
defined function conmutatividad_conjuncion
defined function conmutatividad_disyuncion
defined function distribucion
defined function alternativa
defined function rtr
defined function modusTollens
defined function modusTollensNOT
defined function rImp
defined function rExp
defined function rcontr1
defined function rcontr1_1
defined function rcontr1_2
defined function rcontr2
defined function ria1
defined function ria1_1
defined function ria2
defined function ria2_2
defined function rDf1
defined function rDf1_1
defined function rDf1_2
defined function rDf1_And
defined function rDf2
defined function rDf3_v1
defined function rDf3_v2
defined function rDfAnd
defined function rDfAnd2
defined function rDfEither
defined function rdfNotAnd
defined function rdfNotAndImp
defined function rdfNotAndImp2
defined function rDfImp
defined function reCoimp1
defined function reCoimp2
defined function reCoimp3
defined function reCoimp4
defined trait